In [55]:
# database imports
import re as re
import time
import pandas as pd
import os
#from pymongo import MongoClient # FIX: determine which packages to rmv
import numpy as np

# tokenization imports
#from nltk.corpus import stopwords
import spacy

### Precleaning Notes
- some bullet points don't separate words, so I will need to use regex to find key terms like 'Python' and 'Machine Learning.'
- skills and roles are mentioned multiple times throughout a description, so I will need to find a way to limit the skill count increase to one per page.
- remote positions will mostly be empty, but this does not imply on-site. I should try to check the description as well.
- df.title=df.title.str.lower()
- df.title=df.title.str.lstrip()
- df.title=df.title.str.rstrip()

In [3]:
df = pd.read_csv('linkedin_jobs.csv')

# EDA

In [5]:
df

,title,company,location,post_date,num_applicants,full_time,size,description,salary
0,Data Scientist,Frontiers,Spain,1 week ago,105 applicants,level,"1,001-5,000 employees · Book and Periodical Pu...",About the job\n \n\n \nWe are on a mi...,NaN
1,Senior Data Analyst (f/m),Axpo Group,"Madrid, Community of Madrid, Spain",14 hours ago,1 applicant,Associate,"5,001-10,000 employees · Utilities",About the job\n \n\n \n ...,NaN
2,Data Architect,Adobe,"Barcelona, Catalonia, Spain",1 week ago,56 applicants,level,"10,001+ employees · Software Development",About the job\n \n\n \nOur Company\n\...,NaN
3,Data scientist,HP,"Sant Cugat del Vallès, Catalonia, Spain",1 week ago,30 applicants,level,"10,001+ employees · IT Services and IT Consulting",About the job\n \n\n \n ...,NaN
4,Data Solution Architect,Bumble,"Barcelona, Catalonia, Spain",3 days ago,25 applicants,Associate,"501-1,000 employees · Technology, Information ...",About the job\n \n\n \nWe strongly en...,NaN
...,...,...,...,...,...,...,...,...,...
69,Principal Marketing Data Scientist,Bandai Namco Mobile,"Barcelona, Catalonia, Spain",2 weeks ago,NaN,level,11-50 employees · Computer Games,About the job\n \n\n \nPAC-MAN to Tek...,NaN
70,Data Platform Engineer (f/m/d),Axpo Group,"Madrid, Community of Madrid, Spain",1 day ago,NaN,Associate,"5,001-10,000 employees · Utilities",About the job\n \n\n \n ...,Full-time
71,Global Integration Services Architect,ADP,"Barcelona, Catalonia, Spain",1 day ago,NaN,Contract,"10,001+ employees · Human Resources Services",About the job\n \n\n \nGlobal Integra...,NaN
72,"Data scientist, Engineer, Physicist, Computer ...",German Aerospace Center (DLR),"Almería, Andalusia, Spain",1 month ago,NaN,level,"5,001-10,000 employees · Research Services",About the job\n \n\n \n ...,Part-time · Entry


In [7]:
# replace null values
df.isnull().sum()
#df.dropna(inplace=True)

title              0
company            0
location           0
post_date          1
num_applicants    51
full_time          0
size               0
description        0
salary            53
dtype: int64

**Number of applicants** would be nice to compare against the job's **post date** but so many missing values may skew our data. We will evaluate what we have anyway, knowing this.

**Salary** was difficult to find, but it can be merged with Glassdoor later.

In [21]:
# given the inaccuracy and missing value counts of previous data sets, check columns like salary info...
# and all the rest while we're at it
df.title.value_counts()

Data Analyst                                                                  4
Data Scientist, Commercial Systems                                            3
Senior Data Scientist                                                         3
Data Analysis Specialist                                                      3
Data Architect                                                                2
                                                                             ..
Principal Data Scientist                                                      1
DevOps Engineer (E3)                                                          1
Turkish Seller Onboarding, RCCS                                               1
Python Developer Remote                                                       1
Data scientist, Engineer, Physicist, Computer scientist or similar (f/m/x)    1
Name: title, Length: 63, dtype: int64

### Title Column Tasks
- Use regex to group roles together into new "role column". 
- There are many jobs here that do not match data science, which should be removed.

In [ ]:
# FIX: import data job list from scrape file!

def roles(list_):
    new_list = []
    for item in list_:
        if 'data analyst' in item: # FIX: lower/uppercase doesn't matter
            new_list.append('Data Analyst')
        elif ...
        
    return new_list

df.insert(0, role, roles(df.title)) # FIX: need to self-reference? df = ...

def rm_unrelated_jobs(df):
    for item in df:
        if df.title not in job_list:
            item.drop() # FIX
    return df

df = (rm_unrelated_jobs(df))

In [24]:
df.location.value_counts()

Barcelona, Catalonia, Spain                        39
Madrid, Community of Madrid, Spain                 12
Spain                                               5
Sant Cugat del Vallès, Catalonia, Spain             3
Greater Barcelona Metropolitan Area                 3
Almería, Andalusia, Spain                           2
Alcorcón, Community of Madrid, Spain                1
Seville, Andalusia, Spain                           1
Málaga, Andalusia, Spain                            1
Valencia, Valencian Community, Spain                1
Alicante, Valencian Community, Spain                1
Palma, Balearic Islands, Spain                      1
Zaragoza, Aragon, Spain                             1
European Economic Area                              1
Pamplona, Chartered Community of Navarre, Spain     1
Greater Madrid Metropolitan Area                    1
Name: location, dtype: int64

In [25]:
df.contract_type.value_counts()

level         38
Associate     18
Full-time     14
Internship     1
Executive      1
Platforms      1
Contract       1
Name: full_time, dtype: int64

### Full Time Column Task
- Erroneous entries like 'platforms' should be removed.

In [ ]:
def rm_unrelated_platforms(df):
    contract_types = ['Associate', 'Full-time', 'Internship', 'Executive', 'Contract']
    for item in df:
        if df.contract_type not in contract_types:
            item.drop() # FIX
    return df

In [27]:
df.company_size.value_counts()

AttributeError: 'numpy.int64' object has no attribute 'value_counts'

In [28]:
df.salary.value_counts()

Full-time                 8
Full-time · Entry         7
Full-time · Mid-Senior    3
Part-time · Entry         3
Name: salary, dtype: int64

### Salary Column Task
- As I was not able to grab the correct salary information from LinkedIn, and salary data on LinekdIn is so sparse anyway, I will drop this column and scrape Glassdoor for more info.

In [ ]:
df.drop(['salary'], axis=1, inplace=True)

# Cleaning & Extraction

As **job description** is the most variable and complex column to grab data from, we will have to perform several actions on it and store the new data in a new column, labeled **'skills.'** Unfortunately, bullet points were not captured from the descriptions so some words are joined with the last word of the previous bullet. We will need to add a space before each capital letter just to solve this issue, and the remove any double-spaces.
- basic cleaning (strip, replace line breaks, etc)
- add a space before each capital letter, then remove any double-spaces
- filter out stop words
- tokenize all words
- print description list's most popular terms and save useful words into new column, 'skills'

In [36]:
print(df.description[0])

About the job
          

 
We are on a mission to make science open so everyone can live healthy lives on a healthy planet

Who We Are

Frontiers is an award-winning open science platform and leading open access scholarly publisher.

We are one of the largest and most cited publishers globally. To date, our 200,000 freely available research articles have received more than 1 billion views and downloads and 2 million citations. Our journals span science, health, humanities and social sciences, engineering, and sustainability. And we continue to expand into new academic disciplines so more researchers can publish open access.

Be part of the publishing revolution and help us transform the way research is published, evaluated, and communicated to the world.

Job Role

We are looking for a Data Scientist to work on production-ready Artificial Intelligence (AI) to support our industry-leading AI publishing platform. You will focus on recommender systems and work with Product Owners, ML Eng

In [ ]:
def rm_specials(desc):
    """ remove all special characters from text """
    for char in ['\n','\\','`','*','_','{','}','[',']','(',')','>','#','+','-',',','.','!','$','\'']:
        if char in desc:
            desc = desc.replace(char, "")
    return desc



def clean_text(desc_list):
    """ basic text cleaning before tokenization """
    clean_desc_list = []
    for desc in desc_list:
        desc = desc.strip()
        desc = rm_specials(desc) # remove special characters
        desc = re.sub(r"(\w)([A-Z])", r"\1 \2", desc) # add a space before each capital letter to ensure bullet pointed lists are not joined together
        # desc = desc.replace("  ", " ") # remove double-spaces after capital letter fix above FIX: don't need bc line above does this automatically?
        clean_desc_list.append(desc)
    return clean_desc_list


"""
def tokenize(desc_list): # FIX: how does lemmatization work?
    tokens = []
    nlp = spacy.load("en_core_web_sm")
    tokens = nlp(desc_list)
    
    lemmatized = []
    for token in tokens:
        lemmatized.append(token.lemma_)
    return tokenized
"""


def rm_stops(desc_list):
    """ remove stop words and tokenize descriptions for skill parsing  """
    
    clean_desc_list = [] # save all descriptions here
    
    for desc in desc_list: # loop through each description
        clean_desc = [] # save the given description here, to be added to the clean list after
        for elem in desc.split(" "): # loop through each word in the given description
            if elem not in stop: # skip stopwords
                clean_desc.append(elem) # save words
        stopless_desc = " ".join(clean_desc) # return all words to a string
        clean_desc_list.append(stopless_desc) # add all cleaned descriptions to list
        #tokenized = tokenize(clean_desc_list) # tokenize all words
    return clean_desc_list#tokenized
      


def clean_text(desc_list):
    """ Clean all descriptions for use in parsing to find skills """
    
    nlp = spacy.load("en_core_web_sm")
    stop = nlp.Defaults.stop_words # all stop words, for removing stop words from descriptions

    desc_list = clean_text(fake_list)
    desc_list = rm_stops(fake_list)
    
    return clean_desc_list



def skill_parse(list_):
    """ parse through description for all relevant skills  """
    return # new column



fake_list = ["WordWordWWWWWWWord", 'end of bulletNext bullet', 'we Do We want Data AnalystsDO NOT expect anything', ' with an ML Model!']
fake_list = clean_text(fake_list)
fake_list = rm_stops(fake_list)
print(fake_list)
#print(clean_text(fake_list))

In [ ]:
def sentiment(desc_list):

    # SQL -> DF
    #df = pd.read_sql_query(
    #"""
    #SELECT reviewerName, helpful, reviewText, overall, summary FROM reviews ORDER BY helpful DESC LIMIT 50;
    #""", engine)
    
    # init vars to use in function
    word_list = []
    sentiment_ready = []
    lemmatized = []
    nlp = spacy.load("en_core_web_sm")
    stop = nlp.Defaults.stop_words # all stop words, for removing stop words from reviews

    # find the sentiment of each review
    for desc in desc_list: # for every review
        for word in desc_to_split.split(" "): # for every word in selected review
            if word not in stop:
                word_list.append(word)  # add split words from each review into a list of words (word_list)

        string_without_stop = " ".join(word_list) # turn back into a string
        tokens = nlp(string_without_stop) # tokenize each review

        for token in tokens:
            lemmatized.append(token.lemma_) # lemmatize each review
        sentiment_ready.append(TextBlob(" ".join(lemmatized)).sentiment.polarity) #store the sentiment of each review in a list

    # compare to overall (overall-sentiment*5) & add to df
    sentiment_comparison = []
    for index, rows in df.iterrows():
        # print(rows['overall'], sentiment_ready[index]) # a test print, to visualize the difference before outputing a sum
        sentiment_comparison.append(rows['overall']-sentiment_ready[index]*5) # original review score (out of 5) - new score from NLP analysis of review

    # store comparison and new value back into df for later visualization
    dict_to_merge = {'sentiment polarity':sentiment_ready, 
                    'sentiment comparison':sentiment_comparison} # Add all lists to a new (temp) dataframe.

    # concatenate dataframes
    df2 = pd.DataFrame(dict_to_merge)
    df_new = pd.concat([df, df2], axis=1)

    # export df to new csv
    df_new.to_csv('output/amazon_review_sentiment.csv', index="False")

    print('average difference between user\'s review score (out of 5) vs review sentiment score: ',
        mean(sentiment_comparison))
    
    # df to dict
    new_dict = df_new.to_dict(orient="records")
    
    # add average of sentiment score to top of list returned in API call
    new_dict.insert(0, f'average difference between user\'s review score (out of 5) vs review sentiment score: {mean(sentiment_comparison)}')

    return jsonify(new_dict)


In [ ]:
# tokenize description for skill parsing
df.insert(10, skills, tokenize(df.description)) # FIX: need to self-reference? df =
# FIX # print new list of words
skills_list = [] # write a list of relevant skills to keep

skill_parse(df.description) # parse through description for all relevant skills
df

# Export Data to CSV for Visualization

In [ ]:
df.to_csv('linkedin_jobs_cleaned.csv', index = False, encoding='utf-8')

In [ ]:
# CODE FROM A SEPARATE USER'S GITHUB PROJECT, used for selecting job roles
# https://github.com/AtefehHezavehei/Linkedin_Scraping_Jobs/blob/main/4-data-title-cleaning.ipynb

#TECH

#Frontend
df.loc[df["title"].str.contains("front", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("wordpress", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("php", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("xamarin", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("angular", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("node.js", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("kotlin", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("shopify", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("aem developer", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("golang", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("drupal", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("go developer", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("react", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("ruby", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("swift", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("vue.js", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("rpa developer", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("html", case=False), "title"] = "Frontend_Developer"
df.loc[df["title"].str.contains("devops", case=False), "title"] = "Frontend_Developer"

#Backend
df.loc[df["title"].str.contains("back", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("nodejs", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("magento", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("integration", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("api developer", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("laravel", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("liferay", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("solidity", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("unity", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("abap", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("apigee", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("stack", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("web developer", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("desarrollador", case=False), "title"] = "Backend_Developer"
df.loc[df["title"].str.contains("hardware", case=False), "title"] = "Backend_Developer"

#Data
df.loc[df["title"].str.contains("data", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("datos", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("machine", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("intelligence", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("inteligencia", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("tableau", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("analyst", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("bi engineer", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("analysis", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("analytics", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("business consultant", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("analista", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("power bi", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("scraping", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("bi developer", case=False), "title"] = "Data_Analytics"
df.loc[df["title"].str.contains("analytical", case=False), "title"] = "Data_Analytics"
 

#Security
df.loc[df["title"].str.contains("ciberseguridad", case=False), "title"] = "Cybersecurity"
df.loc[df["title"].str.contains("cybersecurity", case=False), "title"] = "Cybersecurity"
df.loc[df["title"].str.contains("cyber", case=False), "title"] = "Cybersecurity"
df.loc[df["title"].str.contains("security", case=False), "title"] = "Cybersecurity"
df.loc[df["title"].str.contains("seguridad", case=False), "title"] = "Cybersecurity"
df.loc[df["title"].str.contains("virtualización", case=False), "title"] = "Cybersecurity"



#Cloud
df.loc[df["title"].str.contains("cloud", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("azure", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("microsoft", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("aws", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("google", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("Salesforce", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("sales force consultant", case=False), "title"] = "Cloud_Expert"
df.loc[df["title"].str.contains("crm", case=False), "title"] = "Cloud_Expert"


#UX/UI
df.loc[df["title"].str.contains("ux/ui", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("designer", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("diseñador", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("ui ux web design", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("diseño", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("ui developer", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("ecommerce", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("e-commerce", case=False), "title"] = "UX/UI_Designer"
df.loc[df["title"].str.contains("comercio", case=False), "title"] = "UX/UI_Designer"
 
#System Admin
df.loc[df["title"].str.contains("arquitecto", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("administrator", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("database", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("system engineer", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("sistemas", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("systems", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("system", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("linux", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("windows", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("office", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("platform", case=False), "title"] = "System_Admin"
df.loc[df["title"].str.contains("platforma", case=False), "title"] = "System_Admin"


#SW_App Developer
df.loc[df["title"].str.contains("java", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("software", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("python", case=False), "title"] = "SW_APP_Developer" 
df.loc[df["title"].str.contains(".net", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("net", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("programador", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("automation", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("automatización", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("developer senior", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("junior developer", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("sw", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("microservices", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("android", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("aplicaciones", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("ios developer", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("application", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("mobile", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("app", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("ios engineer", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("flutter", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("movilidad", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("smart", case=False), "title"] = "SW_APP_Developer"
df.loc[df["title"].str.contains("smartphone", case=False), "title"] = "SW_APP_Developer"


#Network_Expert
df.loc[df["title"].str.contains("network", case=False), "title"] = "Network_Expert"
df.loc[df["title"].str.contains("telecomunicaciones", case=False), "title"] = "Network_Expert"
df.loc[df["title"].str.contains("telecommunications", case=False), "title"] = "Network_Expert"
df.loc[df["title"].str.contains("redes", case=False), "title"] = "Network_Expert"


#QA Engineer
df.loc[df["title"].str.contains("qa", case=False), "title"] = "QA_Engineer"
df.loc[df["title"].str.contains("test", case=False), "title"] = "QA_Engineer"
df.loc[df["title"].str.contains("quality assurance", case=False), "title"] = "QA_Engineer"
df.loc[df["title"].str.contains("software quality", case=False), "title"] = "QA_Engineer"
df.loc[df["title"].str.contains("tester", case=False), "title"] = "QA_Engineer"


#IT Support
df.loc[df["title"].str.contains("it manager", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("engineering", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("it support", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("it operational", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("technical support", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("internal", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("informatico", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("informática", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("informático", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("sharepoint", case=False), "title"] = "IT_Support"
df.loc[df["title"].str.contains("365", case=False), "title"] = "IT_Support" 

#Solution_Architect
df.loc[df["title"].str.contains("solution", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("solutions", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("architect", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("architecture", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("blockchain", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("tech lead", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("integración", case=False), "title"] = "Solution_Architect"
df.loc[df["title"].str.contains("integrador", case=False), "title"] = "Solution_Architect"
